# AutoPos

In [41]:
from AutoPos.AutoPosModel import AutoPosClass
from sklearn.feature_selection import chi2
from IndModels.ModelsEdited import NGModel,Model,GAACModel

from sklearn.model_selection import train_test_split
import helper
import pandas as pd

import multiprocessing as mp
import numpy as np

In [4]:
Aligned_Data_File = 'Data/Enzyme_aligned.txt'
df = pd.read_csv(Aligned_Data_File,header=None)
enz_names = df[0].values
X = df.iloc[:,1].values
y = df.iloc[:,-1].values
X_train, X_test, y_train, y_test,enz_train,enz_test = train_test_split(X, y,enz_names, test_size=0.25, random_state=7)
sc_func = chi2
n_positions=50
apmodel = AutoPosClass(X_train,X_test,y_train,y_test,sc_func,None,n_positions,imp=True,verbose=False)



In [5]:
apmodel.SVMobject.acc_test

0.5925925925925926

# IndModels

In [20]:
enz_datafile = 'Data/Enzyme-Dataset-Modified.csv'
df = pd.read_csv(enz_datafile,header=None)
enz_names = df[0].values
X = df.iloc[:,1].values
y = df.iloc[:,-1].values


X_train, X_test, y_train, y_test,enz_train,enz_test = train_test_split(X, y,enz_names, test_size=0.25, random_state=7)

m = Model(X_train,X_test,y_train,y_test)

ngmodel = NGModel(X_train,X_test,y_train,y_test)



gmodel = GAACModel(X_train,X_test,y_train,y_test)

ngmodel.SVMobject.acc_test,gmodel.SVMobject.acc_test

(0.7931034482758621, 0.8275862068965517)

In [18]:
def model_evaluate(random_seed):
    Aligned_Data_File = 'Data/Enzyme_aligned.txt'
    df = pd.read_csv(Aligned_Data_File,header=None)
    enz_names = df[0].values
    X = df.iloc[:,1].values
    y = df.iloc[:,-1].values
    X_train, X_test, y_train, y_test,enz_train,enz_test = train_test_split(X, y,enz_names, test_size=0.25, random_state=random_seed)
    sc_func = chi2
    n_positions=50
    apmodel = AutoPosClass(X_train,X_test,y_train,y_test,sc_func,None,n_positions,imp=False,verbose=False)
    
    enz_datafile = 'Data/Enzyme-Dataset-Modified.csv'
    df_k = pd.read_csv(enz_datafile,header=None)
    enz_names_k = df_k[0].values
    X_k = df_k.iloc[:,1].values
    y_k = df_k.iloc[:,-1].values
    X_train_k, X_test_k, y_train_k, y_test_k,enz_train_k,enz_test_k = train_test_split(X_k, y_k,enz_names_k, test_size=0.25, random_state=random_seed)
    ngmodel = NGModel(X_train_k,X_test_k,y_train_k,y_test_k)
    gmodel = GAACModel(X_train_k,X_test_k,y_train_k,y_test_k)
    
    return apmodel.SVMobject.acc_test,ngmodel.SVMobject.acc_test,gmodel.SVMobject.acc_test


In [19]:
model_evaluate(3)

(0.7407407407407407, 0.7931034482758621, 0.7931034482758621)

In [23]:
mp.cpu_count()

8

In [24]:
pool = mp.Pool(mp.cpu_count())

In [25]:
%%time
model_accs = list(pool.map(model_evaluate,range(100)))

CPU times: user 8.34 ms, sys: 6.99 ms, total: 15.3 ms
Wall time: 8.39 s


In [39]:
def get_model_stats(accs):
    mnames = ['AutoPos','NG','GAANG']
    maccs = [[accs[i][j] for i in range(len(accs)) ] for j in range(3)]
    return {mnames[i]:{'min':min(maccs[i]),'mean':np.mean(maccs[i]),
            'max':max(maccs[i]),'std':np.std(maccs[i])} for i in range(len(mnames))}

In [43]:
results_dict = get_model_stats(model_accs)

In [44]:
results_df = pd.DataFrame(results_dict).T

In [46]:
results_df.to_csv('Data/SimResults/myFeatResults.csv',index=True)